<a href="https://colab.research.google.com/github/Priyanka22062001/Priyanka_INFO5731_Spring2025/blob/main/INFO5731_Assignment_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [ ]:
# Install dependencies
!apt-get update
!apt-get install -y wget unzip libvulkan1 xvfb

# Install Google Chrome
!wget -q -O chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i chrome.deb || apt-get install -fy

# Verify Chrome installation
!google-chrome --version

# Install Selenium and WebDriver Manager
!pip install selenium webdriver-manager

# Import necessary modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run headless
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0")

# Initialize WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to IMDb reviews page
driver.get("https://www.imdb.com/title/tt0468569/reviews/")

# Scroll and extract reviews
reviews = []
while len(reviews) <= 1000:
    elements = driver.find_elements("xpath", "//div[@class='ipc-html-content-inner-div']")
    reviews.extend([e.text.strip() for e in elements if e.text.strip()])

    # Scroll and load more reviews
    try:
        load_more = driver.find_element("xpath", "//button//span[contains(text(), '25 more')]")
        driver.execute_script("arguments[0].click();", load_more)
        time.sleep(3)
    except:
        break  # Exit if no more reviews to load

# Close browser
driver.quit()

# Save to CSV
pd.DataFrame(reviews, columns=["review"]).to_csv('imdb_reviews.csv', index=False)

print(f"Scraped {len(reviews)} reviews and saved to 'imdb_reviews.csv'.")


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,610 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Get:13 http://security.ubuntu.com/ubuntu j

In [ ]:
import pandas as pd

df = pd.read_csv("imdb_reviews.csv")
df

,review
0,Best movie ever. Heath ledger's work is phenom...
1,This movie is a work of art. The finest sequel...
2,It is just what you want for the best movie. G...
3,"Confidently directed, dark, brooding, and pack..."
4,We've been subjected to enormous amounts of hy...
...,...
1206,Seeing it rated as the third best movie of all...
1207,This is the future comic book adaptations are ...
1208,"Christopher Nolan's ""The Dark Knight"" isn't me..."
1209,This movie is only great because of Joker and ...


# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load the CSV file
df = pd.read_csv('imdb_reviews.csv')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Get the list of stopwords
stop_words = set(stopwords.words('english'))

# Function to clean the text
def clean_text(text):
    # (1) Remove noise, such as special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # (2) Remove numbers
    text = re.sub(r'\d+', '', text)

    # (3) Remove stopwords
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])

    # (4) Lowercase all texts
    text = text.lower()

    # (5) Stemming
    text = ' '.join([stemmer.stem(word) for word in text.split()])

    # (6) Lemmatization
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return text

# Apply the cleaning function to the 'review' column
df['cleaned_review'] = df['review'].apply(clean_text)

# Save the cleaned data to a new CSV file
df.to_csv('cleaned_imdb_reviews.csv', index=False)

# Display the first few rows of the cleaned data
print(df[['review', 'cleaned_review']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


                                              review  \
0  Best movie ever. Heath ledger's work is phenom...   
1  This movie is a work of art. The finest sequel...   
2  It is just what you want for the best movie. G...   
3  Confidently directed, dark, brooding, and pack...   
4  We've been subjected to enormous amounts of hy...   

                                      cleaned_review  
0     best movi ever heath ledger work phenomen word  
1  movi work art finest sequel ever made dont thi...  
2  want best movi great stori great act thrill tw...  
3  confid direct dark brood pack impress action s...  
4  weve subject enorm amount hype market dark kni...  


# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
import pandas as pd
import spacy
from collections import Counter
from spacy import displacy

# Load the dataset
df = pd.read_csv("/content/cleaned_imdb_reviews.csv")

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Function to extract POS tags
def get_pos_counts(text):
    doc = nlp(text)
    return Counter(token.pos_ for token in doc if token.pos_ in {"NOUN", "VERB", "ADJ", "ADV"})

# Apply POS tagging to the dataset
df["pos_tags"] = df["cleaned_review"].dropna().apply(get_pos_counts)

# Select an example review for parsing analysis
example_text = df["cleaned_review"].dropna().iloc[0]
doc_example = nlp(example_text)

# Render dependency parsing tree
displacy.render(doc_example, style="dep", jupyter=True)

# Function to extract named entity counts
def get_named_entities(text):
    doc = nlp(text)
    return Counter(ent.label_ for ent in doc.ents)

# Apply NER to the dataset
df["named_entities"] = df["cleaned_review"].dropna().apply(get_named_entities)

# Save results to CSV
df.to_csv("imdb_reviews_analysis.csv", index=False)

# Display first few rows
print(df.head())


                                              review  \
0  Best movie ever. Heath ledger's work is phenom...   
1  This movie is a work of art. The finest sequel...   
2  It is just what you want for the best movie. G...   
3  Confidently directed, dark, brooding, and pack...   
4  We've been subjected to enormous amounts of hy...   

                                      cleaned_review  \
0     best movi ever heath ledger work phenomen word   
1  movi work art finest sequel ever made dont thi...   
2  want best movi great stori great act thrill tw...   
3  confid direct dark brood pack impress action s...   
4  weve subject enorm amount hype market dark kni...   

                                        pos_tags  \
0     {'ADJ': 1, 'NOUN': 4, 'ADV': 1, 'VERB': 1}   
1   {'NOUN': 15, 'VERB': 6, 'ADJ': 11, 'ADV': 9}   
2    {'VERB': 4, 'ADJ': 5, 'NOUN': 10, 'ADV': 1}   
3    {'ADJ': 4, 'NOUN': 12, 'VERB': 5, 'ADV': 1}   
4  {'VERB': 16, 'ADJ': 26, 'NOUN': 63, 'ADV': 4}   

             

# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize WebDriver
service = Service(ChromeDriverManager().install())
wd = webdriver.Chrome(service=service, options=chrome_options)

# Data storage lists
product_data = []

# Pagination loop
for page in range(1, 51):
    # Navigate to GitHub Marketplace with pagination
    wd.get(f"https://github.com/marketplace?page={page}&type=actions")
    time.sleep(3)

    # Collect all product links from the current page
    product_links = WebDriverWait(wd, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/marketplace/actions/']"))
    )

    # Extract hrefs from product links
    product_hrefs = [link.get_attribute("href") for link in product_links]

    # Loop through each product href and extract its description
    for product_href in product_hrefs:
        wd.get(product_href)
        time.sleep(3)

        try:
            # Extract product name from URL
            product_name = product_href.split("/")[-1].replace("-", " ")
            print(f"Scraping: {product_name} - {product_href}")

            # Extract product description from markdown-body
            description_element = WebDriverWait(wd, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".markdown-body"))
            )

            # Get full description text
            full_text = description_element.text.strip()

            # Clean the description by joining lines and removing \n (newlines)
            description_cleaned = ' '.join(full_text.split('\n'))

            # Add data to list
            product_data.append([product_name, product_href, description_cleaned])

        except Exception as e:
            print(f"Error scraping description for {product_href}: {e}")
            product_data.append([product_href.split("/")[-1].replace("-", " "), product_href, "Description not found"])

# Close the browser session
wd.quit()

# Create a DataFrame from the data
df2 = pd.DataFrame(product_data, columns=["Product Name", "Product URL", "Description"])

# Save the DataFrame to a CSV file
df2.to_csv("github_marketplace_products.csv", index=False)

# Display the DataFrame
print("\nDataframe with the extracted product information:")
print(df2)

Scraping: trufflehog oss - https://github.com/marketplace/actions/trufflehog-oss
Scraping: metrics embed - https://github.com/marketplace/actions/metrics-embed
Scraping: yq portable yaml processor - https://github.com/marketplace/actions/yq-portable-yaml-processor
Scraping: super linter - https://github.com/marketplace/actions/super-linter
Scraping: gosec security checker - https://github.com/marketplace/actions/gosec-security-checker
Scraping: rebuild armbian and kernel - https://github.com/marketplace/actions/rebuild-armbian-and-kernel
Scraping: opencommit improve commits with ai - https://github.com/marketplace/actions/opencommit-improve-commits-with-ai
Scraping: checkout - https://github.com/marketplace/actions/checkout
Scraping: ssh remote commands - https://github.com/marketplace/actions/ssh-remote-commands
Scraping: github pages action - https://github.com/marketplace/actions/github-pages-action
Scraping: cache - https://github.com/marketplace/actions/cache
Scraping: generate sn

In [ ]:
df2

,Product Name,Product URL,Description
0,trufflehog oss,https://github.com/marketplace/actions/truffle...,TruffleHog Find leaked credentials. 🔎 Now Scan...
1,metrics embed,https://github.com/marketplace/actions/metrics...,📊 Metrics Generate metrics that can be embedde...
2,yq portable yaml processor,https://github.com/marketplace/actions/yq-port...,yq a lightweight and portable command-line YAM...
3,super linter,https://github.com/marketplace/actions/super-l...,Super-Linter Super-linter is a ready-to-run co...
4,gosec security checker,https://github.com/marketplace/actions/gosec-s...,gosec - Go Security Checker Inspects source co...
...,...,...,...
995,ghas policy as code,https://github.com/marketplace/actions/ghas-po...,GitHub Policy as Code Overview GitHub's Policy...
996,notify microsoft teams,https://github.com/marketplace/actions/notify-...,Notify Microsoft Teams Teams notify action ins...
997,install a binary from github releases,https://github.com/marketplace/actions/install...,install-gh-release GitHub Action This reposito...
998,generate sitemap,https://github.com/marketplace/actions/generat...,generate-sitemap Check out all of our GitHub A...


In [ ]:
!pip install nltk

In [ ]:
import nltk

# Download all required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')  # Explicitly download punkt_tab


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Load the CSV file into a DataFrame
file_path = '/content/github_marketplace_products.csv'
df2 = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df2.head())

# Ensure the 'Page Number' column exists
if 'Page Number' not in df2.columns:
    df2['Page Number'] = (df2.index // 20) + 1

# Define the preprocessing function
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    # Converting to Lowercase
    tokens = [word.lower() for word in tokens]
    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a single string
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

# Apply the preprocessing function to the text column
# Assuming the text column is named 'Description'
df2['cleaned_description'] = df2['Description'].apply(preprocess_text)

# Display the first few rows of the DataFrame with cleaned text and Page Number
print(df2[['Description', 'cleaned_description', 'Page Number']].head())

# Save the updated DataFrame back to the same CSV file
df2.to_csv(file_path, index=False)
print(f"Updated data saved to {file_path}")

                 Product Name  \
0              trufflehog oss   
1               metrics embed   
2  yq portable yaml processor   
3                super linter   
4      gosec security checker   

                                         Product URL  \
0  https://github.com/marketplace/actions/truffle...   
1  https://github.com/marketplace/actions/metrics...   
2  https://github.com/marketplace/actions/yq-port...   
3  https://github.com/marketplace/actions/super-l...   
4  https://github.com/marketplace/actions/gosec-s...   

                                         Description  
0  TruffleHog Find leaked credentials. 🔎 Now Scan...  
1  📊 Metrics Generate metrics that can be embedde...  
2  yq a lightweight and portable command-line YAM...  
3  Super-Linter Super-linter is a ready-to-run co...  
4  gosec - Go Security Checker Inspects source co...  
                                         Description  \
0  TruffleHog Find leaked credentials. 🔎 Now Scan...   
1  📊 Metrics Generate 

In [ ]:
df2.head()

,Product Name,Product URL,Description,Page Number,cleaned_description
0,trufflehog oss,https://github.com/marketplace/actions/truffle...,TruffleHog Find leaked credentials. 🔎 Now Scan...,1,trufflehog find leaked credentials . 🔎 scannin...
1,metrics embed,https://github.com/marketplace/actions/metrics...,📊 Metrics Generate metrics that can be embedde...,1,📊 metrics generate metrics embedded everywhere...
2,yq portable yaml processor,https://github.com/marketplace/actions/yq-port...,yq a lightweight and portable command-line YAM...,1,"yq lightweight portable command-line yaml , js..."
3,super linter,https://github.com/marketplace/actions/super-l...,Super-Linter Super-linter is a ready-to-run co...,1,super-linter super-linter ready-to-run collect...
4,gosec security checker,https://github.com/marketplace/actions/gosec-s...,gosec - Go Security Checker Inspects source co...,1,gosec - go security checker inspects source co...


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from urllib.parse import urlparse

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the CSV file into a DataFrame
file_path = '/content/github_marketplace_products.csv'  # Replace with your CSV file path
df2 = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df2.head())

# Ensure the 'Page Number' column exists
if 'Page Number' not in df2.columns:
    df2['Page Number'] = (df2.index // 20) + 1

# Define the preprocessing function
def preprocess_text(text):
  # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove unnecessary whitespace
    text = re.sub(r'\s+', ' ', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Converting to Lowercase
    tokens = [word.lower() for word in tokens]
    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    # Join tokens back into a single string
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

# Apply the preprocessing function to the text column
df2['cleaned_description'] = df2['Description'].apply(preprocess_text)

# Data Quality Checks
# 1. Completeness: Check for missing values and handle them
required_columns = ['Product Name', 'Product URL', 'Description', 'Page Number']
df2.dropna(subset=required_columns, inplace=True)

# 2. Consistency: Ensure data is formatted correctly
# Check if 'Product Name' and 'Description' are strings
df2['Product Name'] = df2['Product Name'].astype(str)
df2['Description'] = df2['Description'].astype(str)

# Check if 'Product URL' is a valid URL
def is_valid_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

# Filter out rows with invalid URLs
df2 = df2[df2['Product URL'].apply(is_valid_url)]

# 3. Accuracy: Remove duplicates
df2.drop_duplicates(inplace=True)

# Display the first few rows of the DataFrame with cleaned text and Page Number
print(df2[['Product Name', 'Product URL', 'Description', 'cleaned_description', 'Page Number']].head())
# Save the updated DataFrame back to the same CSV file
df2.to_csv(file_path, index=False)
print(f"Updated data saved to {file_path}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                 Product Name  \
0              trufflehog oss   
1               metrics embed   
2  yq portable yaml processor   
3                super linter   
4      gosec security checker   

                                         Product URL  \
0  https://github.com/marketplace/actions/truffle...   
1  https://github.com/marketplace/actions/metrics...   
2  https://github.com/marketplace/actions/yq-port...   
3  https://github.com/marketplace/actions/super-l...   
4  https://github.com/marketplace/actions/gosec-s...   

                                         Description  Page Number  \
0  TruffleHog Find leaked credentials. 🔎 Now Scan...            1   
1  📊 Metrics Generate metrics that can be embedde...            1   
2  yq a lightweight and portable command-line YAM...            1   
3  Super-Linter Super-linter is a ready-to-run co...            1   
4  gosec - Go Security Checker Inspects source co...            1   

                                 cleaned_descript

In [ ]:
df2.isnull().sum()

,0
Product Name,0
Product URL,0
Description,0
Page Number,0
cleaned_description,0


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [84]:
!pip install tweepy pandas nltk

In [85]:
import tweepy
import pandas as pd
import time
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# NLTK data
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [86]:
import tweepy

# Keys and tokens
API_KEY = "A9XM4CUUxiv1EREVmrHI8d6p2"
API_SECRET = "SwFHbe7KCxVq4LnPNmbPB5gdC5gNoL8wrlT5xohtfjfilPL02V"
ACCESS_TOKEN = "1891368043550523392-B73A6c2pcq4eOyWu5WnpJNX6sR1QfE"
ACCESS_SECRET = "zAxBnd1H2cUo1c3kMWWwipRNkgxV5XsC4SMwZgzCTJfAM"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAAFSzQEAAAAA2R3F0aAZwUeffItrLy425q6z3fA%3DdY8Qw6IInFXo7QLYvVhukPKkQDX8eFjY0GrWhe9H6NYGaPD6J8"


# Authentication using OAuth 1.0a
auth = tweepy.OAuth1UserHandler(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

client = tweepy.Client(bearer_token=BEARER_TOKEN)

print("✅ Authentication Successful!")

✅ Authentication Successful!


In [ ]:
import tweepy
import pandas as pd
import time
import os

# Initializing Tweepy Client with Authentication
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Function to scrape tweets
def scrape_tweets(query, num_tweets=100):
    tweets_data = []

    try:
        for tweet in tweepy.Paginator(
            client.search_recent_tweets,
            query=query,
            tweet_fields=["id", "text", "created_at", "author_id"],
            max_results=50
        ).flatten(limit=num_tweets):

            tweets_data.append({
                "Tweet_ID": tweet.id,
                "Author_ID": tweet.author_id,
                "Text": tweet.text,
                "Created_At": tweet.created_at
            })

    except tweepy.TooManyRequests:
        print("🔴 API Rate Limit Reached! Waiting 15 minutes before retrying...")
        time.sleep(900)
        return scrape_tweets(query, num_tweets)
    except tweepy.TweepyException as e:
        print(f"🔴 An error occurred: {e}")
        return pd.DataFrame()

    return pd.DataFrame(tweets_data)

# Scraping tweets related to AI & Machine Learning
df_tweets = scrape_tweets("#MachineLearning OR #ArtificialIntelligence -is:retweet", num_tweets=50)

#  Saving tweets to a CSV file
df_tweets.to_csv("AI_ML_Tweets.csv", index=False)

print("✅ Successfully collected & saved 50 tweets in 'AI_ML_Tweets.csv'!")

🔴 API Rate Limit Reached! Waiting 15 minutes before retrying...


In [ ]:
# Loading Raw Tweets
df = pd.read_csv("AI_ML_Tweets.csv")
df.head()

In [ ]:
import pandas as pd
import re

# Loading the raw tweets dataset
df = pd.read_csv("AI_ML_Tweets.csv")

# Step 1: Removing Retweets
df = df[~df["Text"].str.startswith("RT @")]  # Remove rows where text starts with "RT @"

# Step 2: Handling Missing Values
df.dropna(inplace=True)  # Drop rows with missing values

# Step 3: Removing Duplicates
df.drop_duplicates(subset=["Tweet_ID"], inplace=True)  # Drop duplicate tweets based on Tweet_ID

# Step 4: Cleaning Text
def clean_text(text):
    # Removing URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    # Removing mentions (@username)
    text = re.sub(r"@\w+", "", text)
    # Removing hashtags (#)
    text = re.sub(r"#\w+", "", text)
    # Removing newlines and extra spaces
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["Text"] = df["Text"].apply(clean_text)

# Step 5: Converting `Created_At` to DateTime
df["Created_At"] = pd.to_datetime(df["Created_At"])

# Step 6: Data Quality Check
print("\nCleaned Data:")
print(df.head())
print("\nData Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())
print("\nDuplicate Rows:")
print(df.duplicated().sum())

# Saving the cleaned data to a new CSV file
df.to_csv("Cleaned_AI_ML_Tweets.csv", index=False)
print("\n✅ Successfully cleaned and saved the data to 'Cleaned_AI_ML_Tweets.csv'!")

In [ ]:
from google.colab import files
files.download("Cleaned_AI_ML_Tweets.csv")

Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

This assignment was both insightful and challenging. The most difficult part was handling large text data efficiently, especially with constituency and dependency parsing, which required significant computational resources. Troubleshooting missing NLP resources also took extra time. The retirement platform builds its structure with Java and Jakarta Multipart Parser and Apache Struts but must be carefully handled to avoid critical security threats. System integrity along with user data faces serious threats because of inappropriate access controls and file upload vulnerabilities with potential remote code execution. Security enhancements require implementation of Role-based access controls (RBAC) and file validation standards and use of current software patches along with alternative frameworks like Spring Boot.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog